In [110]:
import pandas as pd

In [105]:
df = pd.read_csv("/data/raw_stock_ind.csv")

In [106]:
#import json
#df["info_json_cln"] = df["info_json"].str.replace("\"", " ")
#df["info_json_cln"] = df["info_json_cln"].str.replace("\'", "\"")
#df["info_json_cln"] = df["info_json_cln"].str.replace("None", "\"None\"")

In [107]:
def json_parser(x):
    try:
        return json.loads(x)["symbol"]
    except:
        #print(x)
        return ""

In [108]:
df["company_symbol"] = df["info_json"].map(lambda x: json_parser(x))  

In [109]:
df["company_symbol"].unique()

array([], dtype=object)